In [7]:
# Imports
import os
os.chdir("/projectnb/labci/Lucia/edu-bayesian-MCMC/")

import torch
import matplotlib.pyplot as plt
import pymc as pm
from pymc import PolyaGamma as PG

from src.samplers.MetropolisGibbs import gibbs
from src.samplers.PolyaGamma import polyagamma

torch.manual_seed(42)

In [8]:
# Simulate data
I = 100   # exam items
P = 1000  # pupils

init_a = torch.zeros(I)  
init_b = torch.zeros(I)  
init_t = torch.zeros(P)  
init_W = torch.zeros(P, I)


true_a = torch.empty(size=(I,))      # items' discriminatory power
true_b = torch.empty(size=(I,))      # items' difficulty
true_theta = torch.empty(size=(P,))  # students' skills
true_W = torch.empty(size=(P,I))     # latent variable (for PG case)
Y = torch.empty(size=(P, I))         # simulated data

# populate a, b:
for i in range(I):
    U = torch.distributions.Uniform(torch.tensor([0.0]), torch.tensor([1.0]))
    true_a[i] = U.sample()
    true_b[i] = -1.0*U.sample()
    
# populate theta:
for p in range(P):
    U = torch.distributions.Uniform(torch.tensor([0.0]), torch.tensor([1.0]))
    true_theta[p] = U.sample()

# populate W:
scales = torch.mul(torch.t(true_a[:, None]), true_theta[:, None]) + true_b
true_W = torch.from_numpy(pm.draw(PG.dist(h=1, z=scales)))

# generate exam data:
for i in range(I):
    for p in range(P):
        prob = torch.exp(true_a[i]*true_theta[p] + true_b[i]) / (1 + torch.exp(true_a[i]*true_theta[p] + true_b[i]))
        B = torch.distributions.Bernoulli(prob)
        Y[p][i] = B.sample()

In [ ]:
# Run samplers
N = 1000
sigmasq_a, sigmasq_b, sigmasq_t = torch.tensor([1.0]), torch.tensor([1.0]), torch.tensor([1.0])  # priors

samples_mwg = gibbs(init_a, init_b, init_t, Y, sigmasq_a, sigmasq_b, sigmasq_t, niter=N, adapt=False)
samples_ada_mwg = gibbs(init_a, init_b, init_t, Y, sigmasq_a, sigmasq_b, sigmasq_t, niter=N, adapt=True)
samples_pg = polyagamma(init_a, init_b, init_t, init_W, Y, sigmasq_a, sigmasq_b, sigmasq_t, niter=N)

Starting Gibbs sampler... 
--------------------------------------------



  1%|          | 8/999 [01:55<4:01:12, 14.60s/it]